In [1]:
from __future__ import print_function
import argparse
import torch
#import foolbox
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms, utils
from sklearn.decomposition import PCA
import pickle
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA
import argparse
from numpy import ma
import scipy
import os.path
from sklearn.linear_model import LogisticRegression
import pandas as pd
from skimage.measure import compare_ssim as ssim
from skimage import feature
from skimage import color
from skimage import io

from network.cifar10_cnn_net import cifar10_cnn_net
from network.cifar10_ann import ann_net
from network.adv_cifar10_cnn_net import adv_cifar10_cnn_net
from network.adv_cifar10_ann import adv_ann_net
import utils.cw_final as cw_final
import utils.cw as cw

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



def train_binary(args, model, device, train_loader, optimizer, epoch, target_class):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        batch_size = target.numpy().shape[0]
        
        ## canny edge detection stub
        #print(data.size())
        edge_data = torch.zeros(batch_size, 1024)
        for i in range(batch_size):
            img = color.rgb2gray(data[i].permute(1,2,0).numpy())
            edge_data[i] = torch.Tensor(feature.canny(img, sigma=1.8).astype(float)).reshape(1024)
        
        #data = data.reshape(-1, 784) # Remove this line while training cnn
        edge_data, target = edge_data.to(device), target.to(device)
        
        #print(size)
        for i in range(batch_size):
            if target[i] != target_class:
                #print(target[i])
                target[i] = 1
                #print(target[i])
            else:
                target[i] = 0
        optimizer.zero_grad()
        output = model(edge_data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    
def test_binary(args, model, device, test_loader, target_class):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            batch_size = data.size(0)
            #canny edge detection stub
            edge_data = torch.zeros(batch_size, 1024)
            for i in range(batch_size):
                img = color.rgb2gray(data[i].permute(1,2,0).numpy())
                edge_data[i] = torch.Tensor(feature.canny(img, sigma=1.8).astype(float)).reshape(1024)
                if target[i].data != target_class:
                    target[i] = torch.Tensor([1]).type(torch.LongTensor)#.to(torch.LongTensor)
                else:
                    target[i] = torch.Tensor([0]).type(torch.LongTensor)
            
            #data = data.reshape(data.size(0), 784) # Remove this line while training cnn
                
            edge_data, target = edge_data.to(device), target.to(device)
            output = model(edge_data)
            #print(output)
            #test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1)#, keepdim=True)  # get the index of the max log-probability            
            #print(pred[0])
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [2]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=100, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=50, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=True,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=True,
                    help='For Saving the current Model')
args = parser.parse_args([])
use_cuda = not args.no_cuda and torch.cuda.is_available() 

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0,), (1,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data/cifar10', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0,), (1,))
    ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


Files already downloaded and verified


In [3]:
model = cifar10_cnn_net().to(device)
if (args.save_model):
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
        for epoch in range(1, args.epochs + 1):
            train(args, model, device, train_loader, optimizer, epoch)
            test(args, model, device, test_loader)

        torch.save(model.state_dict(), "cifar_cnn.pt")


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.304314
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.302871
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.302790
Train Epoch: 1 [3840/50000 (8%)]	Loss: 2.301800
Train Epoch: 1 [5120/50000 (10%)]	Loss: 2.303603
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.300408
Train Epoch: 1 [7680/50000 (15%)]	Loss: 2.301327
Train Epoch: 1 [8960/50000 (18%)]	Loss: 2.300168
Train Epoch: 1 [10240/50000 (20%)]	Loss: 2.305464
Train Epoch: 1 [11520/50000 (23%)]	Loss: 2.300316
Train Epoch: 1 [12800/50000 (26%)]	Loss: 2.299764
Train Epoch: 1 [14080/50000 (28%)]	Loss: 2.300703
Train Epoch: 1 [15360/50000 (31%)]	Loss: 2.299283
Train Epoch: 1 [16640/50000 (33%)]	Loss: 2.298994
Train Epoch: 1 [17920/50000 (36%)]	Loss: 2.305095
Train Epoch: 1 [19200/50000 (38%)]	Loss: 2.299613
Train Epoch: 1 [20480/50000 (41%)]	Loss: 2.289013
Train Epoch: 1 [21760/50000 (43%)]	Loss: 2.286434
Train Epoch: 1 [23040/50000 (46%)]	Loss: 2.308280
Train Epoch: 1 [24320/50000 (49%)]	Loss: 2.259104
Train Epoch: 1 

Train Epoch: 5 [1280/50000 (3%)]	Loss: 1.256204
Train Epoch: 5 [2560/50000 (5%)]	Loss: 1.430235
Train Epoch: 5 [3840/50000 (8%)]	Loss: 1.295646
Train Epoch: 5 [5120/50000 (10%)]	Loss: 1.528411
Train Epoch: 5 [6400/50000 (13%)]	Loss: 1.062987
Train Epoch: 5 [7680/50000 (15%)]	Loss: 1.399984
Train Epoch: 5 [8960/50000 (18%)]	Loss: 1.164109
Train Epoch: 5 [10240/50000 (20%)]	Loss: 1.306787
Train Epoch: 5 [11520/50000 (23%)]	Loss: 1.379874
Train Epoch: 5 [12800/50000 (26%)]	Loss: 1.305320
Train Epoch: 5 [14080/50000 (28%)]	Loss: 1.427940
Train Epoch: 5 [15360/50000 (31%)]	Loss: 1.353997
Train Epoch: 5 [16640/50000 (33%)]	Loss: 1.310753
Train Epoch: 5 [17920/50000 (36%)]	Loss: 1.296644
Train Epoch: 5 [19200/50000 (38%)]	Loss: 1.247611
Train Epoch: 5 [20480/50000 (41%)]	Loss: 1.413227
Train Epoch: 5 [21760/50000 (43%)]	Loss: 1.277117
Train Epoch: 5 [23040/50000 (46%)]	Loss: 1.212161
Train Epoch: 5 [24320/50000 (49%)]	Loss: 1.360201
Train Epoch: 5 [25600/50000 (51%)]	Loss: 1.442638
Train Epoc

Train Epoch: 9 [2560/50000 (5%)]	Loss: 1.080789
Train Epoch: 9 [3840/50000 (8%)]	Loss: 1.117066
Train Epoch: 9 [5120/50000 (10%)]	Loss: 1.166809
Train Epoch: 9 [6400/50000 (13%)]	Loss: 0.789091
Train Epoch: 9 [7680/50000 (15%)]	Loss: 0.784632
Train Epoch: 9 [8960/50000 (18%)]	Loss: 1.038311
Train Epoch: 9 [10240/50000 (20%)]	Loss: 1.026783
Train Epoch: 9 [11520/50000 (23%)]	Loss: 0.861893
Train Epoch: 9 [12800/50000 (26%)]	Loss: 0.919193
Train Epoch: 9 [14080/50000 (28%)]	Loss: 0.942335
Train Epoch: 9 [15360/50000 (31%)]	Loss: 1.215655
Train Epoch: 9 [16640/50000 (33%)]	Loss: 0.929619
Train Epoch: 9 [17920/50000 (36%)]	Loss: 0.904319
Train Epoch: 9 [19200/50000 (38%)]	Loss: 0.798887
Train Epoch: 9 [20480/50000 (41%)]	Loss: 0.887000
Train Epoch: 9 [21760/50000 (43%)]	Loss: 1.050922
Train Epoch: 9 [23040/50000 (46%)]	Loss: 1.174572
Train Epoch: 9 [24320/50000 (49%)]	Loss: 1.007993
Train Epoch: 9 [25600/50000 (51%)]	Loss: 1.097071
Train Epoch: 9 [26880/50000 (54%)]	Loss: 0.830029
Train Ep

Train Epoch: 13 [0/50000 (0%)]	Loss: 0.835475
Train Epoch: 13 [1280/50000 (3%)]	Loss: 0.671123
Train Epoch: 13 [2560/50000 (5%)]	Loss: 0.901063
Train Epoch: 13 [3840/50000 (8%)]	Loss: 0.808044
Train Epoch: 13 [5120/50000 (10%)]	Loss: 0.607531
Train Epoch: 13 [6400/50000 (13%)]	Loss: 0.762858
Train Epoch: 13 [7680/50000 (15%)]	Loss: 0.576421
Train Epoch: 13 [8960/50000 (18%)]	Loss: 0.619133
Train Epoch: 13 [10240/50000 (20%)]	Loss: 0.793245
Train Epoch: 13 [11520/50000 (23%)]	Loss: 0.643455
Train Epoch: 13 [12800/50000 (26%)]	Loss: 0.631381
Train Epoch: 13 [14080/50000 (28%)]	Loss: 0.770615
Train Epoch: 13 [15360/50000 (31%)]	Loss: 0.716891
Train Epoch: 13 [16640/50000 (33%)]	Loss: 0.980993
Train Epoch: 13 [17920/50000 (36%)]	Loss: 0.725279
Train Epoch: 13 [19200/50000 (38%)]	Loss: 0.620968
Train Epoch: 13 [20480/50000 (41%)]	Loss: 0.683710
Train Epoch: 13 [21760/50000 (43%)]	Loss: 0.719421
Train Epoch: 13 [23040/50000 (46%)]	Loss: 0.862302
Train Epoch: 13 [24320/50000 (49%)]	Loss: 0.71

Train Epoch: 16 [31200/50000 (100%)]	Loss: 0.612636

Test set: Average loss: 0.6561, Accuracy: 7789/10000 (78%)

Train Epoch: 17 [0/50000 (0%)]	Loss: 0.539122
Train Epoch: 17 [1280/50000 (3%)]	Loss: 0.652237
Train Epoch: 17 [2560/50000 (5%)]	Loss: 0.546303
Train Epoch: 17 [3840/50000 (8%)]	Loss: 0.788773
Train Epoch: 17 [5120/50000 (10%)]	Loss: 0.587173
Train Epoch: 17 [6400/50000 (13%)]	Loss: 0.515758
Train Epoch: 17 [7680/50000 (15%)]	Loss: 0.896271
Train Epoch: 17 [8960/50000 (18%)]	Loss: 0.594016
Train Epoch: 17 [10240/50000 (20%)]	Loss: 0.622761
Train Epoch: 17 [11520/50000 (23%)]	Loss: 0.746550
Train Epoch: 17 [12800/50000 (26%)]	Loss: 0.686316
Train Epoch: 17 [14080/50000 (28%)]	Loss: 0.618330
Train Epoch: 17 [15360/50000 (31%)]	Loss: 0.666698
Train Epoch: 17 [16640/50000 (33%)]	Loss: 0.552463
Train Epoch: 17 [17920/50000 (36%)]	Loss: 0.646103
Train Epoch: 17 [19200/50000 (38%)]	Loss: 0.564814
Train Epoch: 17 [20480/50000 (41%)]	Loss: 0.508727
Train Epoch: 17 [21760/50000 (43%)]

...

Train Epoch: 32 [34560/50000 (69%)]	Loss: 0.412249
Train Epoch: 32 [35840/50000 (72%)]	Loss: 0.388814
Train Epoch: 32 [37120/50000 (74%)]	Loss: 0.550283
Train Epoch: 32 [38400/50000 (77%)]	Loss: 0.515633
Train Epoch: 32 [39680/50000 (79%)]	Loss: 0.229021
Train Epoch: 32 [40960/50000 (82%)]	Loss: 0.539960
Train Epoch: 32 [42240/50000 (84%)]	Loss: 0.365861
Train Epoch: 32 [43520/50000 (87%)]	Loss: 0.400813
Train Epoch: 32 [44800/50000 (90%)]	Loss: 0.568287
Train Epoch: 32 [46080/50000 (92%)]	Loss: 0.330748
Train Epoch: 32 [47360/50000 (95%)]	Loss: 0.478736
Train Epoch: 32 [48640/50000 (97%)]	Loss: 0.377005
Train Epoch: 32 [31200/50000 (100%)]	Loss: 0.458499

Test set: Average loss: 0.5571, Accuracy: 8164/10000 (82%)

Train Epoch: 33 [0/50000 (0%)]	Loss: 0.401583
Train Epoch: 33 [1280/50000 (3%)]	Loss: 0.373778
Train Epoch: 33 [2560/50000 (5%)]	Loss: 0.461989
Train Epoch: 33 [3840/50000 (8%)]	Loss: 0.469868
Train Epoch: 33 [5120/50000 (10%)]	Loss: 0.373147
Train Epoch: 33 [6400/50000 (13%

Train Epoch: 36 [30720/50000 (61%)]	Loss: 0.412027
Train Epoch: 36 [32000/50000 (64%)]	Loss: 0.371078
Train Epoch: 36 [33280/50000 (66%)]	Loss: 0.437768
Train Epoch: 36 [34560/50000 (69%)]	Loss: 0.392737
Train Epoch: 36 [35840/50000 (72%)]	Loss: 0.471284
Train Epoch: 36 [37120/50000 (74%)]	Loss: 0.406598
Train Epoch: 36 [38400/50000 (77%)]	Loss: 0.240245
Train Epoch: 36 [39680/50000 (79%)]	Loss: 0.484589
Train Epoch: 36 [40960/50000 (82%)]	Loss: 0.643899
Train Epoch: 36 [42240/50000 (84%)]	Loss: 0.440247
Train Epoch: 36 [43520/50000 (87%)]	Loss: 0.463461
Train Epoch: 36 [44800/50000 (90%)]	Loss: 0.312921
Train Epoch: 36 [46080/50000 (92%)]	Loss: 0.396949
Train Epoch: 36 [47360/50000 (95%)]	Loss: 0.378519
Train Epoch: 36 [48640/50000 (97%)]	Loss: 0.423793
Train Epoch: 36 [31200/50000 (100%)]	Loss: 0.598286

Test set: Average loss: 0.5839, Accuracy: 8070/10000 (81%)

Train Epoch: 37 [0/50000 (0%)]	Loss: 0.428786
Train Epoch: 37 [1280/50000 (3%)]	Loss: 0.350440
Train Epoch: 37 [2560/50000

Train Epoch: 40 [26880/50000 (54%)]	Loss: 0.418273
Train Epoch: 40 [28160/50000 (56%)]	Loss: 0.162346
Train Epoch: 40 [29440/50000 (59%)]	Loss: 0.368168
Train Epoch: 40 [30720/50000 (61%)]	Loss: 0.255523
Train Epoch: 40 [32000/50000 (64%)]	Loss: 0.325653
Train Epoch: 40 [33280/50000 (66%)]	Loss: 0.425619
Train Epoch: 40 [34560/50000 (69%)]	Loss: 0.365242
Train Epoch: 40 [35840/50000 (72%)]	Loss: 0.282586
Train Epoch: 40 [37120/50000 (74%)]	Loss: 0.179709
Train Epoch: 40 [38400/50000 (77%)]	Loss: 0.260866
Train Epoch: 40 [39680/50000 (79%)]	Loss: 0.374287
Train Epoch: 40 [40960/50000 (82%)]	Loss: 0.475395
Train Epoch: 40 [42240/50000 (84%)]	Loss: 0.311341
Train Epoch: 40 [43520/50000 (87%)]	Loss: 0.318266
Train Epoch: 40 [44800/50000 (90%)]	Loss: 0.385316
Train Epoch: 40 [46080/50000 (92%)]	Loss: 0.342043
Train Epoch: 40 [47360/50000 (95%)]	Loss: 0.336462
Train Epoch: 40 [48640/50000 (97%)]	Loss: 0.444717
Train Epoch: 40 [31200/50000 (100%)]	Loss: 0.430749

Test set: Average loss: 0.550

Train Epoch: 44 [23040/50000 (46%)]	Loss: 0.264448
Train Epoch: 44 [24320/50000 (49%)]	Loss: 0.249637
Train Epoch: 44 [25600/50000 (51%)]	Loss: 0.264782
Train Epoch: 44 [26880/50000 (54%)]	Loss: 0.281649
Train Epoch: 44 [28160/50000 (56%)]	Loss: 0.341835
Train Epoch: 44 [29440/50000 (59%)]	Loss: 0.272624
Train Epoch: 44 [30720/50000 (61%)]	Loss: 0.237553
Train Epoch: 44 [32000/50000 (64%)]	Loss: 0.289747
Train Epoch: 44 [33280/50000 (66%)]	Loss: 0.165375
Train Epoch: 44 [34560/50000 (69%)]	Loss: 0.463618
Train Epoch: 44 [35840/50000 (72%)]	Loss: 0.244625
Train Epoch: 44 [37120/50000 (74%)]	Loss: 0.266951
Train Epoch: 44 [38400/50000 (77%)]	Loss: 0.281576
Train Epoch: 44 [39680/50000 (79%)]	Loss: 0.249512
Train Epoch: 44 [40960/50000 (82%)]	Loss: 0.297220
Train Epoch: 44 [42240/50000 (84%)]	Loss: 0.347371
Train Epoch: 44 [43520/50000 (87%)]	Loss: 0.330399
Train Epoch: 44 [44800/50000 (90%)]	Loss: 0.423148
Train Epoch: 44 [46080/50000 (92%)]	Loss: 0.242199
Train Epoch: 44 [47360/50000 (9

Train Epoch: 48 [19200/50000 (38%)]	Loss: 0.200427
Train Epoch: 48 [20480/50000 (41%)]	Loss: 0.402102
Train Epoch: 48 [21760/50000 (43%)]	Loss: 0.414704
Train Epoch: 48 [23040/50000 (46%)]	Loss: 0.331562
Train Epoch: 48 [24320/50000 (49%)]	Loss: 0.289910
Train Epoch: 48 [25600/50000 (51%)]	Loss: 0.242483
Train Epoch: 48 [26880/50000 (54%)]	Loss: 0.311593
Train Epoch: 48 [28160/50000 (56%)]	Loss: 0.237368
Train Epoch: 48 [29440/50000 (59%)]	Loss: 0.391158
Train Epoch: 48 [30720/50000 (61%)]	Loss: 0.245379
Train Epoch: 48 [32000/50000 (64%)]	Loss: 0.255671
Train Epoch: 48 [33280/50000 (66%)]	Loss: 0.258564
Train Epoch: 48 [34560/50000 (69%)]	Loss: 0.264008
Train Epoch: 48 [35840/50000 (72%)]	Loss: 0.309399
Train Epoch: 48 [37120/50000 (74%)]	Loss: 0.383825
Train Epoch: 48 [38400/50000 (77%)]	Loss: 0.146643
Train Epoch: 48 [39680/50000 (79%)]	Loss: 0.320384
Train Epoch: 48 [40960/50000 (82%)]	Loss: 0.231394
Train Epoch: 48 [42240/50000 (84%)]	Loss: 0.246481
Train Epoch: 48 [43520/50000 (8

Final Model Accuracy:

Test set: Average loss: 0.5847, Accuracy: 8163/10000 (82%)


In [ ]:
# train binary classifiers

model = ann_net().to(device)
optimizer = None
success = 0
for m in range(10):
    # for training of the cnn-network
    if (args.save_model):
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
        for epoch in range(1, args.epochs + 1):
            train_binary(args, model, device, train_loader, optimizer, epoch, m)
            test_binary(args, model, device, test_loader, m)

        torch.save(model.state_dict(), "models/cifar_ann_canny/cifar_ann_canny_binary_sigma_1.8_"+ str(m) + ".pt")

Accruacy for binary Classifier 0 
Accruacy for binary Classifier 1 
Accruacy for binary Classifier 2 
Accruacy for binary Classifier 3 
Accruacy for binary Classifier 4 
Accruacy for binary Classifier 5 
Accruacy for binary Classifier 6 
Accruacy for binary Classifier 7 
Accruacy for binary Classifier 8 
Accruacy for binary Classifier 9 

In [ ]:
# UD case: 
#       1) create adv imgs for trained model without defence for different confidence values
#       2) Mix with equal number of natural imgs
#       3) Calculate accuracy with defence
conf_list = [0, 10, 20]
for conf in conf_list:
    model = adv_cifar10_cnn_net()#.to(device)
    model.load_state_dict(torch.load("cifar_cnn.pt"))
    model.eval()

    classifier = cifar10_cnn_net()
    classifier.load_state_dict(torch.load("cifar_cnn.pt"))
    classifier.eval()

    d0 = ann_net()
    d1 = ann_net()
    d2 = ann_net()
    d3 = ann_net()
    d4 = ann_net()
    d5 = ann_net()
    d6 = ann_net()
    d7 = ann_net()
    d8 = ann_net()
    d9 = ann_net()
    detectors = [d0, d1,d2,d3,d4,d5,d6,d7,d8,d9]#.to(device)

    for i in range(10):
        detectors[i].load_state_dict(torch.load("models/cifar_ann_canny/mnist_ann_canny_binary_sigma_1.8_"+\
                                                str(i) + ".pt"))
        detectors[i].eval()

    inputs_box = (0.0, 1.0)
    adversary = cw.L2Adversary(targeted=False,
                                   confidence=conf,
                                   search_steps=9,
                                   box=inputs_box,
                                   optimizer_lr=0.01)
    adv_imgs = []
    counter = 0
    
    for data, target in test_loader:
        adv, l2 = adversary(model, data, target, to_numpy=False)
        for i in range(args.test_batch_size):
            if l2[i] != np.inf:    #check if attack was successful
                adv_imgs.append(adv[i].reshape(32, 32).numpy())
        if counter == 10:
            break
        else:
            counter += 1
            
    with open("adv_imgs/cifar/UD_cifar_adv_imgs_cw_conf_"+str(conf)+".pkl", "wb") as fp:   #Pickling
        pickle.dump(adv_imgs, fp)            
    
    print("UD Case")
    print("Confidence: ", conf)
    print("Attack Success Rate: ", len(adv_imgs) / 1000)
    print("Avg l2: ", np.mean(l2))
    
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0

    for i in range(len(adv_imgs)):
            adv_im = adv_imgs[i]

            # canny edge sdetection stub
            #print(adv_im.shape)
            #sigma = 1.8 gives best results

            output = classifier(torch.Tensor(adv_im).reshape(1, 3, 32, 32))
            #print(output)
            pred = output.argmax(dim=1, keepdim=True)
            #print(pred)
            adv_im = color.rgb2gray(data[i].permute(1,2,0).numpy())
            adv_edge = feature.canny(adv_im, sigma=1.8).astype(float)
            det_out = detectors[pred](torch.Tensor(adv_edge).reshape(1, 1024))
            prediction = det_out.argmax(dim=1, keepdim=True)
            if prediction == 0:
                false_pos += 1
            else:
                true_pos += 1
        #print(m)
        #print(success1/len(train_adv_imgs[m]))
    counter = 0
    for data, target in test_loader:

        data, target = data.to(device), target.to(device)
        output = classifier(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        for i in range(args.test_batch_size):
            prediction = np.asscalar(pred[i].numpy())
            #detector.load_state_dict(torch.load("models/mnist_ann_canny/mnist_ann_canny_binary"+ str(pred) + ".pt"))
            #detector.eval()
            if (prediction != np.asscalar(target[i].numpy())): #ignoring natural errors
                continue 

            img = color.rgb2gray(data[i].permute(1,2,0).numpy())
            edge_data = feature.canny(img, sigma=1.8).astype(float)
            output = detectors[prediction](torch.Tensor(edge_data).reshape(1, 1024))
            prediction = output.argmax(dim=1, keepdim=True)
            if prediction == 0:
                true_neg += 1
            else:
                false_neg += 1
        if counter == 10:
            break
        else:
            counter += 1
    print("True Positive: ", true_pos)
    print("False Positive: ", false_pos)
    print("True Negative: ", true_neg)
    print("False Negative: ", false_neg)

In [ ]:
# KD Case: Targeted
conf_list = [0, 10, 20]
for conf in conf_list:
    model = cnn_net()#.to(device)
    model.load_state_dict(torch.load("cifar_cnn.pt"))
    model.eval()

    inputs_box = (0.0, 1.0)
    adversary = cw_final.L2Adversary(targeted=True,
                                   confidence=conf,
                                   search_steps=20,
                                   box=inputs_box,
                                   optimizer_lr=0.005)

    adv_imgs = [[] for i in range(10)]
    l2_norms = [[] for i in range(10)]
    success = 0
    class_counter = 0    

    for target_class in range(10):

        detector = adv_ann_net()#.to(device)
        detector.load_state_dict(torch.load("models/cifar_ann_canny/cifar_ann_canny_binary_sigma_1.8_"+\
                                            str(target_class)+".pt"))
        detector.eval()

        # re define test-loader with batch_size 100
        test_loader = torch.utils.data.DataLoader(
                    datasets.CIFAR10('data/cifar10', train=False, transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0,), (1,))])),
                    batch_size=100, shuffle=True, **kwargs)
        
        counter = 0
        for data, label in test_loader:
            #data = torch.Tensor(feature.canny(data.reshape(28, 28).numpy(), sigma=1.0).astype(float)).reshape(1,1,28,28)
            #print(label)
            #if np.asscalar(target.numpy()) == 0:
            #   continue
            target = torch.Tensor([target_class]*100).type(torch.LongTensor)#.to(torch.LongTensor)
            adv, batch_l2_norm = adversary(model, detector, data, target, to_numpy=False)


            #assert isinstance(adversarial_examples, torch.FloatTensor)
            #assert adv.size() == inputs.size()
            #print(success)
            for i in range(100):
                #print(adv.size())
                if batch_l2_norm[i] != np.inf and label[i] != target_class:
                    #output = model(adv[i].reshape(1,1,28,28))
                    #pred = output.argmax(dim=1, keepdim=True)
                    #print("out ", pred)
                    #if np.asscalar(pred.numpy()) == np.asscalar(target.numpy()[i]):
                    #    print("appended")
                    success += 1
                    adv_imgs[target_class].append(adv[i])
                    l2_norms[target_class].append(batch_l2_norm[i])
                if label[i] == target_class:
                    class_counter += 1
            if counter == 11:
                break
            else:
                counter += 1
            
    print("KD Case, conf")
    print("success rate: ", success/(1100 - class_counter))
    print("Examples processed: ", 1100 - class_counter)
    l = []
    for i in range(10):
        l = l + l2_norms[i]
    print("mean l2 norm:", np.mean(l))

        '''with open("adv_imgs/cifar/cw_cifar_adv_imgs_conf_"+str(conf)+"_test_loader_KD_targeted_sigma_1.8_"+\
                      str(target_class)+".pkl", "wb") as fp:   #Pickling
            pickle.dump(adv_imgs, fp)
        with open("adv_imgs/cifar/cw_cifar_l2_norms_conf_"+str(conf)+"_test_loader_KD_targeted_sigma_1.8_"+\
                      str(target_class)+".pkl", "wb") as fp:   #Pickling
            pickle.dump(adv_imgs, fp) '''

In [ ]:
# KD Case: Untargeted

conf_list = [0, 10, 20]
for conf in conf_list:
    model = cnn_net()#.to(device)
    model.load_state_dict(torch.load("cifar_cnn.pt"))
    model.eval()

    d0 = ann_net()
    d1 = ann_net()
    d2 = ann_net()
    d3 = ann_net()
    d4 = ann_net()
    d5 = ann_net()
    d6 = ann_net()
    d7 = ann_net()
    d8 = ann_net()
    d9 = ann_net()
    detectors = [d0, d1,d2,d3,d4,d5,d6,d7,d8,d9]#.to(device)

    for i in range(10):
        detectors[i].load_state_dict(torch.load("models/cifar_ann_canny/cifar_ann_canny_binary_sigma_1.8_"+ str(i) + ".pt"))
        detectors[i].eval()

    inputs_box = (0.0, 1.0)
    adversary = cw_final.L2Adversary(targeted=False,
                                     confidence=conf,
                                     search_steps=9,
                                     box=inputs_box,
                                     optimizer_lr=0.01)

    adv_imgs = []
    l2_norms = []
    success = 0
    counter = 0
    zero_counter = 0

    for data, target in test_loader:
        #data = torch.Tensor(feature.canny(data.reshape(28, 28).numpy(), sigma=1.0).astype(float)).reshape(1,1,28,28)
        #print(label)
        #if np.asscalar(target.numpy()) == 0:
        #   continue
        #print(counter)
        #target = torch.Tensor([0]*batch_size).type(torch.LongTensor)#.to(torch.LongTensor)
        adv, batch_l2_norm = adversary(model, detectors, data, target, to_numpy=False)


        #assert isinstance(adversarial_examples, torch.FloatTensor)
        #assert adv.size() == inputs.size()
        #print(success)
        for i in range(batch_size):
            #print(adv.size())
            if batch_l2_norm[i] != np.inf:
                #output = model(adv[i].reshape(1,1,28,28))
                #pred = output.argmax(dim=1, keepdim=True)
                #print("out ", pred)
                #if np.asscalar(pred.numpy()) == np.asscalar(target.numpy()[i]):
                #    print("appended")
                success += 1
                adv_imgs.append(adv[i])
                l2_norms.append(batch_l2_norm[i])
        if counter == 10:
            break
        else
            counter += 1
    print("KD Untargeted: conf: ", conf)    
    print("Success Rate: ", success/1000)
    print("Avg l2 Norm: ", np.mean(l2_norms))
    
    with open("adv_imgs/cifar/cw_KD_untargeted_cifar_adv_imgs_conf_"+str(conf)+\
              "_test_loader_canny_sigma_1.8.pkl", "wb") as fp:   #Pickling
        pickle.dump(adv_imgs, fp)
    with open("adv_imgs/cifar/cw_KD_untargeted_cifar_l2_norms_conf_"+\
              str(conf)+"_test_loader_canny_sigma_1.8.pkl", "wb") as fp:   #Pickling
        pickle.dump(adv_imgs, fp)        
